In [56]:
import pandas as pd
import numpy as np
import os
import sys

from datetime import timedelta
# Make `src` dir can be imported
project_root_path = os.path.abspath(os.path.join(os.getcwd(), '..'))  # /aicup-predict-energy-generation
sys.path.append(project_root_path)  


from statsmodels.tsa.holtwinters import SimpleExpSmoothing, ExponentialSmoothing

In [2]:
df_sunlight = pd.read_csv('../data/processed_data/sunlight_simulation_data.csv')
df_sunlight['datetime'] = pd.to_datetime(df_sunlight['datetime'])

In [3]:
df_sunlight.head()

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device
0,2024-03-02 06:06:47,0.0,1020.13,14.18,94.37,22.50,0.0,L10
1,2024-03-02 06:07:47,0.0,1020.10,14.19,94.27,28.33,0.0,L10
2,2024-03-02 06:08:47,0.0,1020.07,14.19,94.35,34.17,0.0,L10
3,2024-03-02 06:09:47,0.0,1020.07,14.19,94.42,35.83,0.0,L10
4,2024-03-02 06:10:47,0.0,1020.04,14.20,94.46,37.50,0.0,L10


In [26]:
df_sunlight['date'] = df_sunlight['datetime'].dt.date
df_sunlight['hour'] = df_sunlight['datetime'].dt.hour
df_sunlight

,datetime,windspeed,pressure,temperature,humidity,sunlight,power,device,hour,new_sunlight,date
0,2024-03-02 06:06:47,0.00,1020.13,14.18,94.37,22.50,0.0,L10,6,22.50,2024-03-02
1,2024-03-02 06:07:47,0.00,1020.10,14.19,94.27,28.33,0.0,L10,6,28.33,2024-03-02
2,2024-03-02 06:08:47,0.00,1020.07,14.19,94.35,34.17,0.0,L10,6,34.17,2024-03-02
3,2024-03-02 06:09:47,0.00,1020.07,14.19,94.42,35.83,0.0,L10,6,35.83,2024-03-02
4,2024-03-02 06:10:47,0.00,1020.04,14.20,94.46,37.50,0.0,L10,6,37.50,2024-03-02
...,...,...,...,...,...,...,...,...,...,...,...
85723,2024-07-21 18:44:27,0.00,1005.26,32.27,67.49,37.50,0.0,L10,18,37.50,2024-07-21
85724,2024-07-21 18:45:27,0.00,1005.24,32.26,67.12,32.50,0.0,L10,18,32.50,2024-07-21
85725,2024-07-21 18:46:27,0.52,1005.25,32.24,67.05,28.33,0.0,L10,18,28.33,2024-07-21
85726,2024-07-21 18:47:27,0.26,1005.27,32.18,67.28,24.17,0.0,L10,18,24.17,2024-07-21


In [27]:
hour_for_sub = [10, 11, 12]
df_sunlight['new_sunlight'] = df_sunlight.apply(lambda x: x['sunlight'] if x['hour'] not in hour_for_sub else None, axis=1)

In [28]:
df_sunlight[df_sunlight['hour'].isin(hour_for_sub)].new_sunlight.value_counts()

Series([], Name: count, dtype: int64)

In [46]:
for date in df_sunlight.date.unique().tolist():
    df_date = df_sunlight[df_sunlight['date'] == date].set_index('datetime')

In [47]:
df_date

,windspeed,pressure,temperature,humidity,sunlight,power,device,hour,new_sunlight,date
datetime,,,,,,,,,,
2024-07-21 05:08:26,1.57,1006.61,27.98,88.04,23.33,0.0,L10,5,23.33,2024-07-21
2024-07-21 05:09:26,1.74,1006.61,27.98,87.96,29.17,0.0,L10,5,29.17,2024-07-21
2024-07-21 05:10:26,1.31,1006.63,27.99,87.73,35.83,0.0,L10,5,35.83,2024-07-21
2024-07-21 05:11:26,1.39,1006.62,28.01,87.46,42.50,0.0,L10,5,42.50,2024-07-21
2024-07-21 05:12:26,1.65,1006.60,28.03,87.59,47.50,0.0,L10,5,47.50,2024-07-21
...,...,...,...,...,...,...,...,...,...,...
2024-07-21 18:44:27,0.00,1005.26,32.27,67.49,37.50,0.0,L10,18,37.50,2024-07-21
2024-07-21 18:45:27,0.00,1005.24,32.26,67.12,32.50,0.0,L10,18,32.50,2024-07-21
2024-07-21 18:46:27,0.52,1005.25,32.24,67.05,28.33,0.0,L10,18,28.33,2024-07-21


In [50]:
one = timedelta(minutes=1)
firstTS = df_date[df_date['hour'] < 10].copy()
secondTS = df_date[df_date['hour'] > 12].copy()

firstTSr = firstTS[::-1].copy()
secondTSr = secondTS[::-1].copy()

indexr = pd.date_range(start=firstTS.index[0], end=secondTS.index[-1], freq='min')

In [51]:
firstTSr

,windspeed,pressure,temperature,humidity,sunlight,power,device,hour,new_sunlight,date
datetime,,,,,,,,,,
2024-07-21 09:59:27,1.13,1005.79,45.06,28.27,18785.00,343.94,L10,9,18785.00,2024-07-21
2024-07-21 09:58:27,1.05,1005.82,45.12,28.92,22085.83,574.20,L10,9,22085.83,2024-07-21
2024-07-21 09:57:27,0.96,1005.82,44.60,30.27,23040.83,667.68,L10,9,23040.83,2024-07-21
2024-07-21 09:56:27,1.05,1005.79,43.95,31.08,22651.67,654.71,L10,9,22651.67,2024-07-21
2024-07-21 09:55:27,0.61,1005.84,43.26,30.54,19766.67,441.97,L10,9,19766.67,2024-07-21
...,...,...,...,...,...,...,...,...,...,...
2024-07-21 05:12:26,1.65,1006.60,28.03,87.59,47.50,0.00,L10,5,47.50,2024-07-21
2024-07-21 05:11:26,1.39,1006.62,28.01,87.46,42.50,0.00,L10,5,42.50,2024-07-21
2024-07-21 05:10:26,1.31,1006.63,27.99,87.73,35.83,0.00,L10,5,35.83,2024-07-21


In [55]:
firstTSr.index = indexr[-len(firstTSr):]
secondTSr.index = indexr[:len(secondTSr)]

In [61]:
firstTSr.new_sunlight.index[-1]

Timestamp('2024-07-21 18:48:26')

In [ ]:
#FORWARD    
es = ExponentialSmoothing(firstTS.new_sunlight,  trend='add', seasonal=None).fit()
forwardPrediction = es.predict(start=firstTS.new_sunlight.index[-1]+one, end=secondTS.index[0]-one)

c:\Users\sean.chang\AppData\Local\anaconda3\envs\aicup-energy-generation\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
c:\Users\sean.chang\AppData\Local\anaconda3\envs\aicup-energy-generation\lib\site-packages\statsmodels\tsa\holtwinters\model.py:918: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  warnings.warn(


KeyError: 'The `start` argument could not be matched to a location related to the index of the data.'

In [ ]:
one = timedelta(minutes=1)
secondTSr = secondTS[::-1].copy()
firstTSr = firstTS[::-1].copy()
indexr = pd.date_range(start=firstTS.index[0], end=secondTS.index[-1], freq='min')
firstTSr.index = indexr[-len(firstTSr):]
secondTSr.index = indexr[:len(secondTSr)]

#FORWARD    
es = ExponentialSmoothing(firstTS,  seasonal_periods=seasonal_periods,seasonal='add').fit()
forwardPrediction = es.predict(start=firstTS.index[-1]+one, end=secondTS.index[0]-one)

NameError: name 'timedelta' is not defined